**Members:** Ethan Wong, Timmy Ren, Mason Shu, Medha Nalamada, Carson Mullen, Bethel Kim

**Morning Cohort**: 11 AM - 1 PM

# Install and Load Libraries

In [1]:
#!pip install eventregistry
from eventregistry import *
import pandas as pd
import json

# Scraping Articles with Event Registry API

In [ ]:
# Creating the API key - Can obtain from: https://newsapi.ai/

# Define the configuration data
config_data = {
    "api_key": "your__api_key_here"
}

# Write the configuration data to a JSON file
with open('config.json', 'w') as config_file:
    json.dump(config_data, config_file, indent=4)

In [6]:
# Load the API key from the JSON file
with open("config.json", "r") as file:
    config = json.load(file)
api_key = config["api_key"]

# Initialize EventRegistry with the API key
er = EventRegistry(apiKey=api_key, allowUseOfArchive=False)

# Define topics to search for
topics = ["Election"]

# Define sources to search for and get their URIs
source_names = source_names = ["NPR", "MSNBC", "AP News", "FOX", "Forbes", "New York Times", "Bloomberg", "USA Today", "Washington Post", "Politico", "Oan", "Breitbart", "Wall Street Journal", "cnn.com"]
source_uris = {source: er.getNewsSourceUri(source) for source in source_names}

# List to store the names of all generated DataFrames
dataframe_names = []

# Loop through each topic
for topic in topics:
    # Get the URI for the concept
    concept_uri = er.getConceptUri(topic)
    
    # List to hold all articles' data for the current topic (across all sources)
    articles_data = []
    
    # Loop through each source individually
    for source_name, source_uri in source_uris.items():
        # Define the query for each topic and source
        q = QueryArticlesIter(
            conceptUri=concept_uri,
            sourceUri=source_uri,
            sourceLocationUri=er.getLocationUri("United States"),  # Only US sources
        )

        # Fetch and accumulate up to 1000 articles for the current topic from this source
        for art in q.execQuery(er, sortBy="date", maxItems=1000):
            articles_data.append({
                "title": art.get("title"),
                "source": art.get("source", {}).get("title"),
                "author": art.get("author"),
                "url": art.get("url"),
                "publishedAt": art.get("dateTime"),
                "content": art.get("body")
            })

    # Create a single DataFrame for the current topic with articles from all sources
    articles_df = pd.DataFrame(articles_data)
    
    # Save the DataFrame to a CSV file
    file_name = f"{topic.replace(' ', '_')}_articles.csv"
    articles_df.to_csv(file_name, index=False)

    # Dynamically set the DataFrame name based on the topic, replacing spaces with underscores
    df_name = f"{topic.replace(' ', '_')}_df"
    globals()[df_name] = articles_df

    # Append the DataFrame name to the list
    dataframe_names.append(df_name)

# Print the list of all generated DataFrame names
print("Generated DataFrames:", dataframe_names)

Generated DataFrames: ['Election_df']


In [7]:
Election_df['source'].value_counts()

source
Fox News                    907
The New York Times          822
AP NEWS                     780
Washington Post             727
POLITICO                    712
USA Today                   507
Breitbart                   506
Forbes                      442
Bloomberg Business          411
MSNBC.com                   259
NPR                         220
One America News Network     88
The Wall Street Journal      46
CNN                          16
Name: count, dtype: int64